### Декораторы
Декораторы — это, по сути, "обёртки", которые дают нам возможность изменить поведение функции, не изменяя её код.
Звучит мудрёно. Посмотрим на пример:

In [ ]:
def my_shiny_new_decorator(function_to_decorate):
    # Внутри себя декоратор определяет функцию-"обёртку". Она будет обёрнута вокруг декорируемой,
    # получая возможность исполнять произвольный код до и после неё.
    def the_wrapper_around_the_original_function():
        print("Я - код, который отработает до вызова функции")
        function_to_decorate() # Сама функция
        print("А я - код, срабатывающий после")
    # Вернём эту функцию
    return the_wrapper_around_the_original_function

In [ ]:
def stand_alone_function():
    print("Я простая одинокая функция, ты ведь не посмеешь меня изменять?")

stand_alone_function()

In [ ]:
# Однако, чтобы изменить её поведение, мы можем декорировать её, то есть просто передать декоратору,
# который обернет исходную функцию в любой код, который нам потребуется, и вернёт новую,
# готовую к использованию функцию:
stand_alone_function_decorated = my_shiny_new_decorator(stand_alone_function)
stand_alone_function_decorated()

In [ ]:
stand_alone_function()

Декораторы могут быть использованы для расширения возможностей функций из сторонних библиотек (код которых мы не можем изменять), или для упрощения отладки (мы не хотим изменять код, который ещё не устоялся).

Также полезно использовать декораторы для расширения различных функций одним и тем же кодом, без повторного его переписывания каждый раз.

### Пример 1. Декоратор для вычисления времени работы функции:

In [ ]:
import time
time.perf_counter()

In [ ]:
# Декоратор 1
def benchmark(func):
    """
    Декоратор, выводящий время, которое заняло
    выполнение декорируемой функции.
    """
    import time
    def wrapper(*args, **kwargs):
        t = time.perf_counter() # Засекли время начала выполнения
        res = func(*args, **kwargs) # Запустили
        print(func.__name__, time.perf_counter() - t) # Засекли время окончания исполнения и вывели время конца- время начала
        return res
    return wrapper

# Декоратор 2
def logging(func):
    """
    Декоратор, который выводит вызовы функции.
    """
    def wrapper(*args, **kwargs):
        res = func(*args, **kwargs)
        print(func.__name__, args, kwargs)
        return res
    return wrapper

# Декоратор 3
def counter(func):
    """
    Декоратор, считающий и выводящий количество вызовов
    декорируемой функции.
    """
    def wrapper(*args, **kwargs):
        wrapper.count += 1
        res = func(*args, **kwargs)
        print ("{0} была вызвана: {1}x".format(func.__name__, wrapper.count))
        return res
    wrapper.count = 0
    return wrapper

In [ ]:
@benchmark 
# @logging
# @counter
def reverse_string(string):
    return ''.join(reversed(string))

In [ ]:
print(reverse_string("А роза упала на лапу Азора"))

Те же самые декораторы можно запустить для любой функции. Например, для функции, которая считает количество вхождений некоторого слова в текст романа Достоевского "Идиот"

In [ ]:
import requests
import re
from random import randint
@benchmark
@logging
@counter
def get_random_idiot_word_count(word):
    the_idiot_url = 'https://www.gutenberg.org/files/2638/2638-0.txt'

    # Отправляем запрос в библиотеку Gutenberg и забираем текст
    raw = requests.get(the_idiot_url).text
    #Заменим в тексте все небуквенные символы на пробелы
    processed_book = re.sub('[\W]+' , ' ', raw).lower()
    return len(re.findall(word.lower(),processed_book))

get_random_idiot_word_count('whole')

Один из суперполезных способов использования декоратора – [мемоизация](https://ru.wikipedia.org/wiki/Мемоизация) aka кэширование. По сути, это способ "переиспользовать" предыдущие результаты вместо того, чтобы по 25 раз считать одно и то же. Рекомендую заглянуть [сюда](https://habr.com/ru/post/335866/)

In [1]:
def memo(f):
    """
    Запомнить результаты исполнения функции f
    """
    cache = {}
    def fmemo(*args):
        if args not in cache:
            cache[args] = f(*args)
        return cache[args]
    return fmemo



Иллюстрацию того, что мемоизация - суперполезная штука, можно произвести на примере расчета чисел Фибоначчи. Если вы программист, они вам, наверняка, уже оскомину набили, но мы все равно на них посмотрим.

In [ ]:
import time
# @memo
def fib(n):
    if n < 2:
        return n
    return fib(n-2) + fib(n-1)

# Какое число мы хотим посчитать
x = 40

t1 = time.perf_counter()
print(f'fib({x}) =', fib(x))
print(time.perf_counter() - t1)

Это ОЧЕНЬ долго. 

In [2]:
import time
@memo
def fib(n):
    if n < 2:
        return n
    return fib(n-2) + fib(n-1)

# Какое число мы хотим посчитать
x = 40

t1 = time.perf_counter()
print(f'fib({x}) =', fib(x))
print(time.perf_counter() - t1)

fib(40) = 102334155
0.00022590000000022314


Как вам ускорение? :)

In [ ]:
t = time.perf_counter()
time.sleep(2)
time.perf_counter() - t

In [ ]:
t1 = time.perf_counter()
print(f'fib({x}) =', fib(x))
print(time.perf_counter() - t1)